# Do the variable relationships between voltage clamp and current clamp differ for different cre lines or layers?

## Objective 
Determine parameters found by fitting current clamp data can be predicted from from parameters found fitting voltage clamp data (using fits to the average first pulse trace).  If so, do these relationships differ between different cre lines or layers?

## Methods
a) fit data <br>
b) rate the quality of the fit <br>
c) choose the goodness of fit one would want to use <br>
c) git rid of heteroscedasticity <br>
d) break into catagories (i.e. inhibitory/excitatory, cre line, layer) <br>
e) test for statistical significance of linear regression <br>
f) test for statistical significance that intercepts and slopes of catagories are different <br>

## Subquestions addressed to perform this analysis
What is the best way to get rid of heteroscedasticity? <br>
How good of a fit do we need to assess? <br>



## Results


In [ ]:
# Compare voltage clamp and current clamp fits.
# Note that these .csv files loaded here have been processed though "extract_first_pulse_fit_data_from_DB.py"
# "and catagorize_goodness_of_fit_by_eye.py"
# A fantastic explanation of how to interpret the output and input of linear regression
# of different catagories in statsmodels (or R) is at: 
# https://www.andrew.cmu.edu/user/achoulde/94842/lectures/lecture10/lecture10-94842.html
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
font={'size':22}
matplotlib.rc('font', **font)
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
# load csv files.
i_df=pd.read_csv('ML_connected_iclamp_2018_12_18.csv')
v_df=pd.read_csv('ML_connected_vclamp_2018_12_12.csv')
i_df['uid']=i_df.apply(lambda row: "%.3f" % float(row.uid), axis=1)
v_df['uid']=v_df.apply(lambda row: "%.3f" % float(row.uid), axis=1)

In [ ]:
# get rid of wacky Unnamed columns if they exist
v_df=v_df[v_df.columns.drop(list(v_df.filter(regex='Unnamed')))]
i_df=i_df[i_df.columns.drop(list(i_df.filter(regex='Unnamed')))]
i_df.keys()

In [ ]:
#Merge the any sign versus forced sign data frames
merged_df = pd.merge(i_df, v_df, on=['uid', 'pre_cell_id', 'post_cell_id', 
                                     'distance', 'acsf','post_cre', 'pre_cre',
                                    'boolean_connection', 'pre_layer', 
                                     'post_layer'], how='inner', suffixes={'_i', '_v'})
merged_df['uid']=merged_df['uid'].astype(str)

# note that the length of the merged data frame equaling the len of the smallest 
# individual dataframe shows that the values being merged on are the same in the
# two databases.
print(len(i_df))
print(len(v_df))
print(len(merged_df))

In [ ]:
merged_df.keys()

In [ ]:
# look at data that is 'excellent'
excellent_df=merged_df[(merged_df['good_fit_i']=='excellent') & (merged_df['good_fit_v']=='excellent') & 
                      (merged_df['data_clarity_v']=='well') & (merged_df['data_clarity_i']=='well')]
# look at data that is pretty good
good_df=merged_df[((merged_df['good_fit_i']=='excellent') | (merged_df['good_fit_i']=='good')) & 
                       ((merged_df['good_fit_v']=='excellent') | (merged_df['good_fit_v']=='good')) & 
                       ((merged_df['data_clarity_v']=='well') | (merged_df['data_clarity_v']=='ok')) & 
                       ((merged_df['data_clarity_i']=='well') | (merged_df['data_clarity_i']=='ok'))]


# look at what is in data sets
print('excellent fits')
print(excellent_df.groupby(['pre_cre', 'post_cre']).size())

# print('\nJust layer 5')
# print(excellent_df[excellent_df['pre_layer']=='5'].groupby(['pre_cre', 'post_cre']).size())

print('\ngood fits')
print(good_df.groupby(['pre_cre', 'post_cre']).size())


# Heteroscedasticity
Look at data and significance tests for heteroscedasticity with different transforms (sqrt and log). <br>
Note that in transforms, the amplitude the negative numbers are converted to positive.  


In [ ]:
def pos_sqrt(n):
    if n<0:
        out = np.sqrt(np.abs(n))
    if n>=0:
        out = np.sqrt(n)
    if np.isnan(n):
        return np.nan
    return out

def pos_log(n):
    if n<0:
        out = np.log(np.abs(n))
    if n>=0:
        out = np.log(n)
    if np.isnan(n):
        return np.nan
    return out

def plot_hetero(df):
    variables=(('NRMSE_v', 'NRMSE_i'),
               ('amp_v', 'amp_i'),
               ('rise_time_v', 'rise_time_i'),
               ('latency_v', 'latency_i'),
               ('decay_tau_v', 'decay_tau_i'))

                              
    for var in variables:
        f=plt.figure(figsize=(20,10))
        # plot data on left
        f.add_subplot(131)
        regression=smf.ols(formula='%s ~ %s' %(var[1], var[0]), data=df).fit() 
        _,_,_,p_unchanged=statsmodels.stats.diagnostic.het_white(regression.resid, regression.model.exog)
        sns.regplot(var[0], var[1], data=df, fit_reg=True, color ='k', label='p=%f' % p_unchanged)
        sns.regplot(var[0], var[1], data=df[df['syn_excitation_v']=='in'], fit_reg=True, color ='r')
        sns.regplot(var[0], var[1], data=df[df['syn_excitation_v']=='ex'], fit_reg=True, color ='g')
        plt.xlim([np.min(df[var[0]]), np.max(df[var[0]])])
        plt.ylim([np.min(df[var[1]]), np.max(df[var[1]])])
        plt.legend()


        # plot square root transformed data on right
        f.add_subplot(132)
        df['sqrt_'+var[0]]=df[var[0]].apply(lambda x: pos_sqrt(x))
        df['sqrt_'+var[1]]=df[var[1]].apply(lambda x: pos_sqrt(x))
        regression=smf.ols(formula='%s ~ %s' %('sqrt_'+var[1], 'sqrt_'+var[0]), data=df).fit() 
        _,_,_,p_sqrt=statsmodels.stats.diagnostic.het_white(regression.resid, regression.model.exog)
        sns.regplot('sqrt_'+var[0], 'sqrt_'+var[1], data=df, color ='k', label='p=%f' % p_sqrt)
        sns.regplot('sqrt_'+var[0], 'sqrt_'+var[1], data=df[df['syn_excitation_v']=='in'], fit_reg=True, color ='r')
        sns.regplot('sqrt_'+var[0], 'sqrt_'+var[1], data=df[df['syn_excitation_v']=='ex'], fit_reg=True, color ='g')
        plt.xlim([np.min(df['sqrt_'+var[0]]), np.max(df['sqrt_'+var[0]])])
        plt.ylim([np.min(df['sqrt_'+var[1]]), np.max(df['sqrt_'+var[1]])])
        plt.legend()

        # plot log transformed data on the far right
        f.add_subplot(133)
        df['log_'+var[0]]=df[var[0]].apply(lambda x: pos_log(x))
        df['log_'+var[1]]=df[var[1]].apply(lambda x: pos_log(x))
        regression=smf.ols(formula='%s ~ %s' %('log_'+var[1], 'log_'+var[0]), data=df).fit() 
        _,_,_,p_log=statsmodels.stats.diagnostic.het_white(regression.resid, regression.model.exog)
        sns.regplot('log_'+var[0], 'log_'+var[1], data=df, fit_reg=True, color ='k', label='p=%f' % p_log)
        sns.regplot('log_'+var[0], 'log_'+var[1], data=df[df['syn_excitation_v']=='in'], fit_reg=True, color ='r')
        sns.regplot('log_'+var[0], 'log_'+var[1], data=df[df['syn_excitation_v']=='ex'], fit_reg=True, color ='g')
        plt.xlim([np.min(df['log_'+var[0]]), np.max(df['log_'+var[0]])])
        plt.ylim([np.min(df['log_'+var[1]]), np.max(df['log_'+var[1]])])
        plt.legend()

    plt.show()
    return df



### Plots for excellent data set

In [ ]:
excellent_df=plot_hetero(excellent_df)

## look at good (larger) data set

In [ ]:
good_df=plot_hetero(good_df)

# Inhibitory significantly different from excitatory?

In [ ]:
# To test if the different catagories are the same we must make new catagorical variables
# Must get rid of data from the data set we don't want (it might be true that statmodels
# skips empty cells but I don't know) and set a catagory value to the rest. 
def catagory_df(df, catagory_type):
    
    if catagory_type=='cre':
        col_name=('pre_cre', 'post_cre')
        values=(('pvalb', 'pvalb'),
                ('rorb', 'rorb'),
                ('sim1', 'sim1'),
                ('tlx3','tlx3'),
                ('unknown', 'unknown'))
    elif catagory_type=='layer':
        col_name=('pre_layer', 'post_layer')
        values=(('2', '2'),
               ('2/3', '2/3'),
               ('3', '3'),
               ('4', '4'),
               ('5', '5'),
               ('6', '6'))
    else:
        raise Exception('catagory doesnt exist')
        
    #for each cre catagory get a subset make a new column for catagory
    new_df=pd.DataFrame()
    for value in values: 
        cat_df=df[(df[col_name[0]]==value[0]) & (df[col_name[1]]==value[1])]
        key=catagory_type+'_catagory'
        v=value[0]+'_to_'+value[1]
        cat_df[key]=v
        # concatenate to whole dataFrame
#         print(new_df)
#         print(cat_df)
        new_df=pd.concat([new_df, cat_df], axis=0, join='outer', join_axes=None, ignore_index=True,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=False)
    return new_df

cre_cat_df=catagory_df(excellent_df, 'cre')
print(cre_cat_df.groupby('cre_catagory').size())
layer_cat_df=catagory_df(excellent_df, 'layer')
print(layer_cat_df.groupby('layer_catagory').size())

In [ ]:
excellent_df.keys()

# Below is an example on using NRMSE in cre lines.  All conditions will be in a section below

In [ ]:
#catagorical stats of regressions of different cre lines note that they match plots
model=smf.ols(formula='NRMSE_i ~ cre_catagory * NRMSE_v', data=cre_cat_df).fit()
model.summary()

In [ ]:
# check to see if the intercept is significantly different
int_same_model=smf.ols(formula='NRMSE_i ~ NRMSE_v', data=cre_cat_df).fit() 
int_diff_model=smf.ols(formula='NRMSE_i ~ cre_catagory + NRMSE_v', data=cre_cat_df).fit()
# I don't understand why the p-value is nan here....
sm.stats.anova_lm(int_diff_model, int_same_model)

In [ ]:
#note the does not perfectly match the regression plot with all the data because only a subset of data is being used here.
int_same_model.summary()

In [ ]:
# check to see if slopes are significantly different
sm.stats.anova_lm(int_diff_model, model)

# Stats for all variable with cre line catagories

In [ ]:
def do_stats(df, catagory_type):
    '''
    df: pandas DataFrame
            should be proccessed from catagory_df
    catagory: string
        specifies which catagory to apply. Options are 'cre' or 'layer'.
    Note: make sure the supplied df corresponds to the supplied catagory_type 
    '''
    if catagory_type=='cre':
        col_name=('cre_catagory')
        print ('TESTING CRE LINE CATAGORIES')
    elif catagory_type=='layer':
        col_name=('layer_catagory')
        print ('TESTING LAYER CATAGORIES')

    else:
        raise Exception('catagory doesnt exist')
    variables=(('NRMSE_v', 'NRMSE_i'),
               ('amp_v', 'amp_i'),
               ('rise_time_v', 'rise_time_i'),
               ('latency_v', 'latency_i'),
               ('decay_tau_v', 'decay_tau_i'))
    for var in variables:
        # each catagory is allowed independent intercepts and slopes
        model=smf.ols(formula='%s ~ %s * %s' %('log_'+var[1], col_name, 'log_'+var[0]), data=df).fit()
        # all data lumped together
        int_same_model=smf.ols(formula='%s ~ %s' %('log_'+var[1], 'log_'+var[0]), data=df).fit() 
        # each catagory is allowed independent intercepts
        int_diff_model=smf.ols(formula='%s ~ %s + %s' %('log_'+var[1],col_name, 'log_'+var[0]), data=df).fit()
        print var
        # Note that simplier model must go first!!!!
        # check to see if the intercepts are significantly different
        print('check if intercepts are different')
        print(sm.stats.anova_lm(int_same_model, int_diff_model))
        # check to see if slopes are significantly different
        print('check if slopes are different')
        print(sm.stats.anova_lm(int_diff_model, model))
        print("\n")
        
do_stats(cre_cat_df, 'cre')
do_stats(layer_cat_df, 'layer')

# Catagory plots

In [ ]:
def catagory_plots(df, plot_type):
    
    if plot_type=='cre':
        col_name=('pre_cre', 'post_cre')
        values=(('pvalb', 'pvalb', 'b'),
                ('rorb', 'rorb', 'r'),
                ('sim1', 'sim1', 'g'),
                ('tlx3','tlx3', 'm'),
                ('unknown', 'unknown', 'c'))
    elif plot_type=='layer':
        col_name=('pre_layer', 'post_layer')
        values=(('2', '2','b'),
               ('2/3', '2/3', 'r'),
               ('3', '3', 'g'),
               ('4', '4', 'm'),
               ('5', '5', 'c'),
               ('6', '6', 'y'))
    else:
        raise Exception()
    
    variables=(('NRMSE_v', 'NRMSE_i'),
               ('amp_v', 'amp_i'),
               ('rise_time_v', 'rise_time_i'),
               ('latency_v', 'latency_i'),
               ('decay_tau_v', 'decay_tau_i'))

                              
    for var in variables:
        fs=15   

        plt.figure(figsize=(fs,fs))
        mod = smf.ols(formula='%s ~ %s' % ('log_'+var[1], 'log_'+var[0]), data=df)
        res=mod.fit()
        sns.regplot('log_'+var[0], 'log_'+var[1], data=df, fit_reg=True, color ='k', 
                    label='all, n=%i, slope=%f, intercept=%.2E' % 
                    (len(df), res.params['log_'+var[0]], res.params.Intercept))
        for value in values: 
            plot_df=df[(df[col_name[0]]==value[0]) & (df[col_name[1]]==value[1])]
            mod = smf.ols(formula='%s ~ %s' % ('log_'+var[1], 'log_'+var[0]), data=plot_df)
            res=mod.fit()
            print(var, value)
            print(res.summary())
            print(res.pvalues)
            print('\n')
            sns.regplot('log_'+var[0], 'log_'+var[1], data=plot_df, fit_reg=True, color=value[2], 
                        label='%s to %s, n=%i, p=%f' % 
                        (value[0], value[1], len(plot_df), res.pvalues[1]))
#             sns.regplot(var[0], var[1], data=plot_df, fit_reg=True, color=value[2], 
#                         label='%s to %s, n=%i, slope=%.2E, int=%.2E' % 
#                         (value[0], value[1], len(plot_df), res.params[var[0]], res.params.Intercept))
        plt.xlim([np.min(df['log_'+var[0]]), np.max(df['log_'+var[0]])])
        plt.ylim([np.min(df['log_'+var[1]]), np.max(df['log_'+var[1]])])
        plt.legend()
        plt.show()
    

## Cre line catagories

In [ ]:
#plot all the data
matplotlib.rc('font', **font)
catagory_plots(excellent_df, 'cre')

## Layer catagories

In [ ]:
matplotlib.rc('font', **font)
catagory_plots(excellent_df, 'layer')

# Show images of the voltage and current clamp best fits used in analysis above 

In [ ]:
from IPython.display import Image, display 
import matplotlib.image as mpimg
for p in excellent_df[['image_path_i', 'image_path_v']].iterrows():
#    if type(p) is str:
    print (p[1].image_path_i)
    print (p[1].image_path_v)    
#     display(Image(filename=p[1].image_path_i, width=400, height=400))
#     display(Image(filename=p[1].image_path_v, width=400, height=400)) 
    f=plt.figure(figsize=(20,10))
    f.add_subplot(121)
    plt.imshow(mpimg.imread(p[1].image_path_i))
    plt.axis('off')
    f.add_subplot(122)
    plt.imshow(mpimg.imread(p[1].image_path_v)) 
    plt.axis('off')
    plt.show()
